In [1]:
import pickle
import warnings
import re
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
import onnx
import tf2onnx
import onnxruntime as ort


from typing import List
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

onnx.__version__

'1.14.0'

Для "облегчения" виртуального окружения исключим необходимость добавлять tensorflow переведя модель в формат onnx

In [2]:
model = tf.keras.models.load_model('../predictors/rnn/model.h5')

In [3]:
onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, '../predictors/rnn/model.onnx')

Проверим результаты конвертации модели

In [4]:
date = datetime.strptime('2023-09-01', '%Y-%m-%d')

start_time = (date - relativedelta(years=1)).replace(
    hour=0,
    minute=0,
    second=0,
    microsecond=0
)

end_time = date.replace(
    hour=23,
    minute=0,
    second=0,
    microsecond=0
)

predict_time = (date - timedelta(days=1)).replace(
    hour=11,
    minute=0,
    second=0,
    microsecond=0
)

conn = sqlite3.connect('../energy_consumpion.sqlite')
cursor = conn.cursor()

query = f"""
    SELECT
        t.power_true,
        t.temperature,
        t.datetime,
        (CASE WHEN
            day_off_table.day_off = 1
        THEN
            1
        ELSE
            0
        END) AS day_off
    FROM
        consumption_table AS t
        LEFT JOIN day_off_table ON DATE(t.datetime) = DATE(day_off_table.datetime)
    WHERE
        t.datetime >= '{start_time}' AND
        t.datetime <= '{end_time}';
"""

data = pd.read_sql(sql=query, con=conn)
conn.close()
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S')

print('data.shape', data.shape)
data.head(2)

data.shape (8784, 4)


,power_true,temperature,datetime,day_off
0,3385.571934,12.686667,2022-09-01 00:00:00,0
1,3295.563483,11.673333,2022-09-01 01:00:00,0


In [5]:
def rnn_model(data: pd.DataFrame,
              date: datetime) -> np.array:
    """
    Функция делает предсказание потребления элетроэнергии на предстоящие сутки вперед
    На вход необходимо передать датафрейм с данными за год
    и момент времени, из которого делается предикт
    11:00 соответствует полудню... Вот такие пироги...
    """
    def make_data(df       : pd.DataFrame,
                  y_lags   : List[int],
                  time_freq: List[str],) -> pd.DataFrame:
        """
        Функция, формирующая исходные для предикта от рекурренной нейронной сети
        На вход требует:

        Датафрейм df, содержащий данные о почасовом потреблении и темпераутре.

        Список лагов, по которым будет осуществляться сдвиг во времени назад.
        Рекомендуется давать начиная с нуля. Последнее значение не включается.

        Список лагов для целевой переменной,
        по которым будет осуществляться сдвиг во времени вперед.
        Рекомендуется давать начиная с единицы. Последнее значение не включается.

        Список частот, по которым необходимо кодировать время и дату.
        Доступны: {'hour', 'day_of_year', 'month', 'weekday'}
        """
        time_dict = {'hour'        : 24,
                     'day_of_year' : 365.25,
                     'month'       : 12,
                     'weekday'     : 7}
        df.set_index('datetime', inplace=True)

        # следующеие фичи нужны для рекуррентного входа

        # извлечем тригонометрическую фичу из времени
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            for time_component in time_freq:
                df[time_component] = getattr(df.index, time_component)
                df[f'{time_component}_sin'] = \
                    np.sin( 2 * np.pi * df[time_component] / time_dict[time_component])
                df[f'{time_component}_cos'] = \
                    np.cos( 2 * np.pi * df[time_component] / time_dict[time_component])
                df.drop([time_component], axis=1, inplace=True)

        # следующеие фичи нужны для полносвязного входа

        # присоединим к полносвязному вектору данные о времени,
        # назначенном для конкретного предсказания, а также инфу о рабочем/выходном дне
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            for lag in y_lags:
                for time_component in time_freq:
                    df[f'{time_component}_sin_{lag}'] = df[f'{time_component}_sin'].shift(-lag - 1)
                    df[f'{time_component}_cos_{lag}'] = df[f'{time_component}_cos'].shift(-lag - 1)
                df[f'day_off_{lag}'] = df['day_off'].shift(-lag - 1)

        # присоединим к вектору данные о прошлогоднем потреблении и температуре в этот же час,
        # а также сведения о выходном/праздничном дне
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            # создадим временный датафрейм, расширенный на год назад,
            # чтобы получить временные сдвиги
            temp_df = pd.concat([pd.DataFrame(columns=df.columns,
                                              index = pd.date_range(start=df.index[0] - \
                                                                    relativedelta(years=1),
                                                                    end=df.index[0],
                                                                    freq='H')[:-1]),
                                 df[['power_true',
                                     'temperature',
                                     'day_off']].copy()
                                ], axis=0)

            for lag in y_lags:
                # получим соответствующие лагу индексы
                shifted_index = tuple(index - \
                                      relativedelta(years=1) + \
                                        relativedelta(hours=lag + 1) for index in df.index)
                # получим значения по временным сдвигам
                df[[f'one_hour_consumption_previos_year_{lag}',
                    f'one_hour_temperature_previos_year_{lag}',
                    f'day_off_previos_year_{lag}']] = \
                        temp_df.loc[pd.DatetimeIndex(shifted_index),
                                    ['power_true',
                                     'temperature',
                                     'day_off']].values

        df.rename(columns={'power_true': 'one_hour_consumption',
                           'temperature': 'one_hour_temperature'},
                  inplace=True)

        return df


    # получим исходные данные для предикта
    y_lags=list(range(12, 36))
    window_size = 179
    time_freq = ['hour', 'day_of_year', 'month', 'weekday']
    X = make_data(df=data,
                  y_lags=y_lags,
                  time_freq=time_freq)

    # для дальнейшей обработки определим перечни колонок

    # рекуррентный вход
    # скаллируемые колонки
    RNN_input_scalled_cols = ['one_hour_consumption', 'one_hour_temperature']

    # нескаллируемые колонки
    RNN_input_not_scalled_regex = r'[a-zA-Z_]+(?:sin|cos|off)\b'
    RNN_input_not_scalled_cols = []

    # полносвязный вход
    # скаллируемые колонки
    Dense_input_scalled_regex = \
        r'(?:one_hour_consumption_previos_year|one_hour_temperature_previos_year)_(\d+)'
    Dense_input_scalled_cols = []

    # нескаллируемые колонки
    Dense_input_not_scalled_regex = \
        r'[a-zA-Z_]+[(?:sin|cos|off)|(?:day_off_previos_year)]_(?=.*\d+)'
    Dense_input_not_scalled_cols = []

    for regex, cols in zip([RNN_input_not_scalled_regex,
                            Dense_input_scalled_regex,
                            Dense_input_not_scalled_regex,],
                           [RNN_input_not_scalled_cols,
                            Dense_input_scalled_cols,
                            Dense_input_not_scalled_cols,]):
        cols.extend([col_name for col_name in X.columns if re.findall(regex, col_name)])

    Dense_input_not_scalled_cols = \
        [item for item in Dense_input_not_scalled_cols if item not in Dense_input_scalled_cols]

    RNN_input_cols = RNN_input_scalled_cols + RNN_input_not_scalled_cols
    Dense_input_cols = Dense_input_scalled_cols + Dense_input_not_scalled_cols

    # выделим из полученных данных матрицу для рекуррентного слоя и
    # вектор для полносвязного слоя
    RNN_input = X.loc[date - timedelta(hours=window_size): date, RNN_input_cols]
    Dense_input = pd.DataFrame(X.loc[date, Dense_input_cols]).astype(float).T

    with open('../predictors/rnn/scallers.pickle', 'rb') as file:
        scaller_RNN, scaller_Dense = pickle.load(file)

    # проскаллируем данные
    RNN_input.loc[:, RNN_input_scalled_cols] = scaller_RNN \
        .transform(RNN_input.loc[:, RNN_input_scalled_cols] \
                   .values.reshape(-1, len(RNN_input_scalled_cols)))
    Dense_input.loc[:, Dense_input_scalled_cols] = scaller_Dense \
        .transform(Dense_input.loc[:, Dense_input_scalled_cols] \
                   .values.reshape(-1, len(Dense_input_scalled_cols)))

    # приведем данные к типу np.array
    RNN_input = np.array(RNN_input).reshape(-1, *RNN_input.shape)
    Dense_input = np.array(Dense_input).reshape(Dense_input.shape)
    Input = [RNN_input, Dense_input]

    return Input


Input = rnn_model(data=data.copy(), date=predict_time)

In [6]:
model = tf.keras.models.load_model('../predictors/rnn/model.h5')
pred = model.predict(Input)
pred

1/1 [==============================] - 0s 231ms/step


array([[3697.5024, 3551.8816, 3448.0122, 3440.0244, 3470.961 , 3603.1265,
        3902.7651, 4280.38  , 4623.8467, 4840.6724, 4913.3574, 4881.355 ,
        4873.673 , 4885.9976, 4851.307 , 4799.7026, 4704.1196, 4608.984 ,
        4572.611 , 4547.775 , 4586.701 , 4547.1353, 4285.034 , 3937.9934]],
      dtype=float32)

In [8]:
session = ort.InferenceSession('../predictors/rnn/model.onnx')
inputDetails = session.get_inputs()
pred = session.run(None, {
    inputDetails[0].name: Input[0].astype(np.float32),
    inputDetails[1].name: Input[1].astype(np.float32),
})
pred

[array([[3697.5024, 3551.882 , 3448.012 , 3440.0242, 3470.9607, 3603.1262,
         3902.7654, 4280.3804, 4623.8467, 4840.673 , 4913.3574, 4881.355 ,
         4873.6733, 4885.998 , 4851.3076, 4799.7026, 4704.12  , 4608.984 ,
         4572.6104, 4547.7744, 4586.7007, 4547.1353, 4285.034 , 3937.994 ]],
       dtype=float32)]